<a href="https://colab.research.google.com/github/yaminiravala/5731/blob/main/Yamini_Exercise_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The third In-class-exercise (due on 11:59 PM 10/08/2023, 40 points in total)

The purpose of this exercise is to understand text representation.

Question 1 (10 points): Describe an interesting text classification or text mining task and explain what kind of features might be useful for you to build the machine learning model. List your features and explain why these features might be helpful. You need to list at least five different types of features.

Your answer here (no code for this question, write down your answer as detail as possible for the above questions):
Sentiment Analysis of Product Reviews

In this text classification task, i aim to build a machine learning model to analyze the sentiment of product reviews, determining whether a review is positive, negative, or neutral. This task is valuable for businesses to understand customer opinions and feedback on their products and make data-driven decisions for improvements.

Some features for the Machine Learning Model:

Bag of Words (BoW) / Term Frequency-Inverse Document Frequency (TF-IDF): BoW or TF-IDF representations capture the frequency of individual words in the text. They help the model understand which words are prominent in positive and negative reviews.

Word Embeddings : It transform words into dense vectors, capturing semantic relationships between words. This helps the model understand context and meaning in reviews.

N-grams: This represent sequences of 'n' adjacent words. They capture the ordering of words and phrases in text, allowing the model to identify more complex patterns in sentiment.

Sentiment Lexicons : This assign sentiment scores to words. By incorporating these scores, the model can identify sentiment-bearing words and their polarities.

Part-of-Speech Tags: POS tagging labels words in a sentence as nouns, verbs, adjectives, etc. Analyzing the distribution of POS tags can reveal syntactic patterns associated with sentiment.

Question 2 (10 points): Write python code to extract these features you discussed above. You can collect a few sample text data for the feature extraction.

In [7]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00


In [14]:
# You code here (Please add comments in the code):
from sklearn.feature_extraction.text import CountVectorizer
# Sample text
sample_text = [
    "This product is amazing! I love it!",
    "The quality of this product is terrible.",
    "I'm neutral about this product. It's okay.",
]

# creating a CountVectorizer instance
vectorizer = CountVectorizer()

# fit the vectorizer to the text and transform the text into a BoW representation
bow_matrix = vectorizer.fit_transform(sample_text)
# get the feature names in the vocabulary
feature_names = vectorizer.get_feature_names_out()
# convert the BoW matrix to a dense array and display the result
bow_array = bow_matrix.toarray()
# print the BoW feature matrix
print("Bag of Words Feature Matrix:")
print(bow_array)

# print the feature names
print("\nFeature Names :")
print(feature_names)


Bag of Words Feature Matrix:
[[0 1 1 1 1 0 0 0 1 0 0 0 1]
 [0 0 1 0 0 0 1 0 1 1 1 1 1]
 [1 0 0 1 0 1 0 1 1 0 0 0 1]]

Feature Names :
['about' 'amazing' 'is' 'it' 'love' 'neutral' 'of' 'okay' 'product'
 'quality' 'terrible' 'the' 'this']


Question 3 (10 points): Use any of the feature selection methods mentioned in this paper "Deng, X., Li, Y., Weng, J., & Zhang, J. (2019). Feature selection for text classification: A review. Multimedia Tools & Applications, 78(3)."

Select the most important features you extracted above, rank the features based on their importance in the descending order.

In [15]:
# You code here (Please add comments in the code):
# here i'm using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# create a TfidfVectorizer instance
tfidf_vectorizer = TfidfVectorizer()

# fit and transform the sample text into a TF-IDF representation
tfidf_matrix = tfidf_vectorizer.fit_transform(sample_text)

# calculate the importance scores (sum of TF-IDF values for each feature)
feature_importance_scores = np.sum(tfidf_matrix, axis=0)

# get the feature names (words in the vocabulary)
feature_names = tfidf_vectorizer.get_feature_names_out()

# creating a dictionary to store feature names and their importance scores
feature_importance_dict = {
    feature_names[i]: feature_importance_scores[0, i]
    for i in range(len(feature_names))
}

# sorting in descending order
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
# print the scores
for feature, importance_score in sorted_features:
    print(f"{feature}: {importance_score:.4f}")

product: 0.8436
this: 0.8436
it: 0.7552
is: 0.7185
amazing: 0.5094
love: 0.5094
about: 0.4836
neutral: 0.4836
okay: 0.4836
of: 0.4354
quality: 0.4354
terrible: 0.4354
the: 0.4354


Question 4 (10 points): Write python code to rank the text based on text similarity. Based on the text data you used for question 2, design a query to match the most relevant docments. Please use the BERT model to represent both your query and the text data, then calculate the cosine similarity between the query and each text in your data. Rank the similary with descending order.

In [16]:
# You code here (Please add comments in the code):
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
#sample query to test
query = "How is the product?"
# here i am loading pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
# tokenize and encode the query
query_tokens = tokenizer(query, padding=True, truncation=True, return_tensors="pt")
query_output = model(**query_tokens).last_hidden_state.mean(dim=1)
#encode all documents
document_tokens = tokenizer(sample_text, padding=True, truncation=True, return_tensors="pt")
document_outputs = model(**document_tokens).last_hidden_state.mean(dim=1)
# calculate cosine similarities
similarities = cosine_similarity(query_output.detach().numpy(), document_outputs.detach().numpy())
# flatten th array.
similarities = similarities.flatten()

# Rank indescending order
sorted_indices = similarities.argsort()[::-1]
sorted_documents = [sample_text[i] for i in sorted_indices]
# print
print("Ranked Documents:")
for i, doc in enumerate(sorted_documents, start=1):
    print(f"Rank {i}: Similarity = {similarities[sorted_indices[i-1]]:.4f}")
    print(doc)
    print()

Ranked Documents:
Rank 1: Similarity = 0.6566
The quality of this product is terrible.

Rank 2: Similarity = 0.6296
I'm neutral about this product. It's okay.

Rank 3: Similarity = 0.6286
This product is amazing! I love it!

